In [1]:
import ccapi

config         = ccapi.Configuration()
config.url     = "http://localhost:5000"
# config.verbose = True

### Building a Constraint-Based Model

This simple example demonstrates how to create a constraint-based model, its metabolites, reactions, genes, etc. into [Cell Collective](https://cellcollective.org).

We’ll use the `3OAS140` reaction from the `STM_1.0 model`:

```
1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] → 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]
```

Begin by importing the ccapi module into your workspace.

In [2]:
import ccapi

Now, let’s try creating a client object in order to interact with services provided by [Cell Collective](https://cellcollective.org).

In [3]:
client = ccapi.Client()

Authenticate your client using a ***password flow type authentication*** scheme.

**NOTE**: *Before you can authenticate using ccapi, you must first register an application of the appropriate type on [Cell Collective](https://cellcollective.org). If you do not require a user context, it is read only.*

In [4]:
def authenticate():
    client.auth(email = "test@cellcollective.org", password = "test")

try:
    authenticate()
except ccapi.exception.AuthenticationError:
    client.sign_up(
        email       = "test@cellcollective.org",
        password    = "test",
        first_name  = "Test",
        last_name   = "Test",
        institution = "Test"
    )
    
    authenticate()

In [5]:
user = client.me()
user

ID,1
Name,Test Test
Memory Address,0x07f1548034f28
First Name,Test
Last Name,Test
Email Address,test@cellcollective.org
Institution,Test


#### Creating a Base Model

Create a Base Model using ccapi and instantize it with an authenticated client.

In [6]:
model = ccapi.Model("Sample Constraint-Based Model", type_ = "metabolic", client = client)
# model.save3()

A `ccapi.Model` consists of various `ccapi.ModelVersion` objects that help you build various versions to a model. By default, a `ccapi.Model` provides you a default model version of a boolean type. You can override the same by passing the `type_`.

In [7]:
# get the default model version
metabolic      = model.versions[0]
metabolic.name = "Version 1"
metabolic

ID,-14721
Version,-38038
Name,Version 1
Memory Address,0x07f1548034eb8
Number of Metabolites,0
Metabolites,
Number of Reactions,0
Reactions,


#### Adding Metabolites to a Constraint-Based Model

First, we need to create a list of metabolite objects for this model.

In [8]:
# create compartment
compartment   = ccapi.Compartment("Cytosol")

# create metabolites

malACP_c      = ccapi.Metabolite("Malonyl-acyl-carrier-protein",
                                 formula     = "C14H22N2O10PRS",
                                 compartment = compartment)
ACP_c         = ccapi.Metabolite("acyl-carrier-protein",
                                 formula     = "C11H21N2O7PRS",
                                 compartment = compartment)
h_c           = ccapi.Metabolite("H",
                                 formula     = "H",
                                 compartment = compartment)
ddcaACP_c     = ccapi.Metabolite("Dodecanoyl-ACP-n-C120ACP",
                                 formula     = "C23H43N2O8PRS",
                                 compartment = compartment)
co2_c         = ccapi.Metabolite("CO2",
                                 formula     = "CO2",
                                 compartment = compartment)
omrsACP_c     = ccapi.Metabolite("3-Oxotetradecanoyl-acyl-carrier-protein",
                                 formula     = "C25H45N2O9PRS",
                                 compartment = compartment)

Now let us add a list of components to our Constraint-Based Model.

In [9]:
metabolic.add_metabolites(malACP_c, ACP_c, h_c, ddcaACP_c, co2_c, omrsACP_c)
metabolic

ID,-14721
Version,-38038
Name,Version 1
Memory Address,0x07f1548034eb8
Number of Metabolites,6
Metabolites,"Malonyl-acyl-carrier-protein, acyl-carrier-protein, H, Dodecanoyl-ACP-n-C120ACP, CO2, 3-Oxotetradecanoyl-acyl-carrier-protein"
Number of Reactions,0
Reactions,


In [10]:
metabolic.compartments

ID,Name
-23107,Cytosol


In [11]:
metabolic.metabolites

ID,Name,Compartment,Formula,Charge
-63265,Malonyl-acyl-carrier-protein,Cytosol,C14H22N2O10PRS,
-6276,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-43564,H,Cytosol,H,
-31239,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-41007,CO2,Cytosol,CO2,
-8271,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


#### Saving a Model

Ensure you save your model in order to commit your work.

In [12]:
# model.save3()

In [13]:
metabolic.metabolites

ID,Name,Compartment,Formula,Charge
-63265,Malonyl-acyl-carrier-protein,Cytosol,C14H22N2O10PRS,
-6276,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-43564,H,Cytosol,H,
-31239,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-41007,CO2,Cytosol,CO2,
-8271,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


#### Adding Reactions to a Model

First, we create a `ccapi.Reaction` object.

In [14]:
reaction = ccapi.Reaction("3 oxoacyl acyl carrier protein synthase n C140",
                         subsystem   = "Cell Envelope Biosynthesis",
                         lower_bound = 0,
                         upper_bound = 1000)

Adding metabolites to a reaction requires using a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [15]:
reaction.add_metabolites({
    malACP_c:  -1,
    h_c:       -1,
    ddcaACP_c: -1,
    co2_c:      1,
    ACP_c:      1,
    omrsACP_c:  1
})

We will now add a reaction to the model

In [16]:
metabolic.add_reaction(reaction)

In [17]:
# model.save3()

In [18]:
metabolic.reactions

ID,Name,Number of Positive Regulators,Number of Negative Regulators,Subsystem,Lower Bound,Upper Bound
-63591,3 oxoacyl acyl carrier protein synthase n C140,0,0,Cell Envelope Biosynthesis,0,1000


#### Adding a Gene Reaction Rule to a Reaction

In order to add a Gene Reaction Rule to a Reaction, create genes as follows.

In [19]:
STM2378 = ccapi.Gene("STM2378")
STM1197 = ccapi.Gene("STM1197")

Let's consider a Gene Reaction Rule as follows

```
STM2378 or STM1197
```

You can add the same using the Regulatory Mechanism provided for Boolean Models as follows:

In [20]:
reaction.add_regulators(
    ccapi.PositiveRegulator(STM2378),
    ccapi.PositiveRegulator(STM1197)
)

In [21]:
# model.save3()

#### Creating a Version

Let's first create a copy of our Model Version object.

In [22]:
copy = metabolic.copy()
copy

ID,-55322
Version,-38038
Name,Version 1
Memory Address,0x07f15480438d0
Number of Metabolites,6
Metabolites,"Malonyl-acyl-carrier-protein, acyl-carrier-protein, H, Dodecanoyl-ACP-n-C120ACP, CO2, 3-Oxotetradecanoyl-acyl-carrier-protein"
Number of Reactions,1
Reactions,3 oxoacyl acyl carrier protein synthase n C140


Let's change any one of the metabolite names.

In [23]:
metabolite      = copy.metabolites[0]
metabolite.name = "foobar"

copy.metabolites

ID,Name,Compartment,Formula,Charge
-12389,foobar,Cytosol,C14H22N2O10PRS,
-48999,acyl-carrier-protein,Cytosol,C11H21N2O7PRS,
-39294,H,Cytosol,H,
-52464,Dodecanoyl-ACP-n-C120ACP,Cytosol,C23H43N2O8PRS,
-34898,CO2,Cytosol,CO2,
-64494,3-Oxotetradecanoyl-acyl-carrier-protein,Cytosol,C25H45N2O9PRS,


Add this new version to your model.

In [24]:
model.add_version(copy)

In [25]:
model.save3()

ID,1
Name,Sample Constraint-Based Model
Memory Address,0x07f1548034da0
Number of Versions,2
Versions,"[<ConstraintBasedModel 1 version 2 at 0x07f1548034eb8 name='Version 1'>, <ConstraintBasedModel 1 version 2 at 0x07f15480438d0 name='Version 1'>]"


#### Dynamic Linking [Experimental]

Let's assume that you'd like to reuse certain components within a model.

In [26]:
model     = ccapi.Model("Test", type_ = "metabolic", client = client)
# model.save3()

In [27]:
metabolic = model.versions[0]
metabolic

ID,-47299
Version,-2636
Name,
Memory Address,0x07f153f3b4be0
Number of Metabolites,0
Metabolites,
Number of Reactions,0
Reactions,


In [28]:
compartment = ccapi.Compartment(compartment_id = 1) # Linking to an existing Compartment
metabolite  = ccapi.Metabolite("test", compartment = compartment)

metabolic.add_metabolite(metabolite)

In [29]:
model.save3()

ID,2
Name,Test
Memory Address,0x07f153f3b4ac8
Number of Versions,1
Versions,[<ConstraintBasedModel 2 version 3 at 0x07f153f3b4be0>]
